## DES

In [4]:
import pyDes
import base64

Key = "12345678"  #加密的key

Iv = None   #偏移量


def bytesToHexString(bs):
    '''
    bytes转16进制
    '''
    return ''.join(['%02X ' % b for b in bs])
def hexStringTobytes(str):
    '''16进制转bytes'''
    str = str.replace(" ", "")
    return bytes.fromhex(str)

# 加密
def encrypt_str(data):
    # 加密方法
    #padmode填充方式
    #pyDes.ECB模式
    method = pyDes.des(Key, pyDes.ECB, Iv, pad=None, padmode=pyDes.PAD_PKCS5)
    # 执行加密码 hex输出
    k = method.encrypt(data)
    data = bytesToHexString(k).replace(' ','')
    #bs64手粗
    #data =base64.b64encode(k)
    return data

# 解密
def decrypt_str(data):
    method = pyDes.des(Key, pyDes.ECB, Iv, pad=None, padmode=pyDes.PAD_PKCS5)
    k =hexStringTobytes(data)
    #bs64
    #k = base64.b64decode(data)
    return method.decrypt(k)


Encrypt = encrypt_str("aaa")
print(Encrypt)
Decrypt = decrypt_str(Encrypt)
print(Decrypt)

3DAE77C81F31CCC3
b'aaa'


## 3DES

In [6]:
import hashlib, base64
import json
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.primitives.ciphers import algorithms
from Crypto.Cipher import DES3

def pkcs7padding(text):
    """
    明文使用PKCS7填充
    最终调用DES3加密方法时，传入的是一个byte数组，要求是16的整数倍，因此需要对明文进行处理
    :param text: 待加密内容(明文)
    :return:
    """
    bs = DES3.block_size  # 16
    length = len(text)
    bytes_length = len(bytes(text, encoding='utf-8'))
    # tips：utf-8编码时，英文占1个byte，而中文占3个byte
    padding_size = length if (bytes_length == length) else bytes_length
    padding = bs - padding_size % bs
    # tips：chr(padding)看与其它语言的约定，有的会使用'\0'
    padding_text = chr(padding) * padding
    return text + padding_text


def pkcs7_unpad(content):
    """
    解密时候用
    :param content:
    :return:
    """
    if not isinstance(content, bytes):
        content = content.encode()
    pad = PKCS7(algorithms.DES3.block_size).unpadder()
    pad_content = pad.update(content) + pad.finalize()
    return pad_content


def encrypt(key, content):
    """
    DES3加密
    key,iv使用同一个
    模式cbc
    填充pkcs7
    :param key: 密钥
    :param content: 加密内容
    :return:
    """
    key_bytes = bytes(key, encoding='utf-8')
    iv = key_bytes
    cipher = DES3.new(key_bytes, DES3.MODE_ECB)
    # 处理明文
    content_padding = pkcs7padding(content)
    # 加密
    encrypt_bytes = cipher.encrypt(bytes(content_padding, encoding='utf-8'))
    # 重新编码
    result = str(base64.b64encode(encrypt_bytes), encoding='utf-8')
    return result

def decrypt(key,text):
    key_bytes = bytes(key, encoding='utf-8')
    iv = key_bytes
    cryptos = DES3.new(key_bytes, DES3.MODE_ECB)
    data = cryptos.decrypt(text)
    return json.loads(pkcs7_unpad(data))

## AES
### AES_ECB_pkcs5padding(该模板不兼容中文)

In [7]:
from Crypto.Cipher import AES
import base64

class Aes_ECB(object):
    def __init__(self):
        self.key = 'XXXXXXXXXXX'  #秘钥
        self.MODE = AES.MODE_ECB
        self.BS = AES.block_size
        self.pad = lambda s: s + (self.BS - len(s) % self.BS) * chr(self.BS - len(s) % self.BS)
        self.unpad = lambda s: s[0:-ord(s[-1])]

    # str不是16的倍数那就补足为16的倍数
    def add_to_16(value):
        while len(value) % 16 != 0:
            value += '\0'
        return str.encode(value)  # 返回bytes

    def AES_encrypt(self, text):
        aes = AES.new(Aes_ECB.add_to_16(self.key), self.MODE)  # 初始化加密器
        encrypted_text = str(base64.encodebytes(aes.encrypt(Aes_ECB.add_to_16(self.pad(text)))),
                             encoding='utf-8').replace('\n', '')  # 这个replace大家可以先不用，然后在调试出来的结果中看是否有'\n'换行符
        # 执行加密并转码返回bytes
        return encrypted_text

In [8]:
import hashlib, base64
from Crypto.Cipher import AES
from cryptography.hazmat.primitives.padding import PKCS7
from cryptography.hazmat.primitives.ciphers import algorithms


def pkcs7padding(text):
    """
    明文使用PKCS7填充
    最终调用AES加密方法时，传入的是一个byte数组，要求是16的整数倍，因此需要对明文进行处理
    :param text: 待加密内容(明文)
    :return:
    """
    bs = AES.block_size  # 16
    length = len(text)
    bytes_length = len(bytes(text, encoding='utf-8'))
    # tips：utf-8编码时，英文占1个byte，而中文占3个byte
    padding_size = length if (bytes_length == length) else bytes_length
    padding = bs - padding_size % bs
    # tips：chr(padding)看与其它语言的约定，有的会使用'\0'
    padding_text = chr(padding) * padding
    return text + padding_text


def pkcs7_unpad(content):
    """
    解密时候用
    :param content:
    :return:
    """
    if not isinstance(content, bytes):
        content = content.encode()
    pad = PKCS7(algorithms.AES.block_size).unpadder()
    pad_content = pad.update(content) + pad.finalize()
    return pad_content


def encrypt(key, content):
    """
    AES加密
    key,iv使用同一个
    模式cbc
    填充pkcs7
    :param key: 密钥
    :param content: 加密内容
    :return:
    """
    key_bytes = bytes(key, encoding='utf-8')
    iv = key_bytes
    cipher = AES.new(key_bytes, AES.MODE_ECB)
    # 处理明文
    content_padding = pkcs7padding(content)
    # 加密
    encrypt_bytes = cipher.encrypt(bytes(content_padding, encoding='utf-8'))
    # 重新编码
    result = str(base64.b64encode(encrypt_bytes), encoding='utf-8')
    return result

def decrypt(key,text):
    key_bytes = bytes(key, encoding='utf-8')
    iv = key_bytes
    cryptos = AES.new(key_bytes, AES.MODE_ECB)
    data = cryptos.decrypt(text)
    return json.loads(pkcs7_unpad(data))